# Grammar Checker using Naive Bayes approach

## Training Data

In [5]:
import os, pprint
pp = pprint.PrettyPrinter(indent=2)
file = "../Data/train.txt"
# print(file)


# Reading File
with open(file) as f:
    lines = f.read().split("\n")
# lines = lines[:3000]
# pp.pprint(lines)

def error_check(x):
    """ To check if a line in training set contains error or not
    Return type is training word , flag(to know if error exist or not)
    and err(list of errors that a ngram can have) """
    wor = ""
    flg = 0
    err = []
    for i in x:
        nw = ""
        for ch in i:
            if ch == ' ':
                flg = 1
                words = i.split()
                err.append(words[len(words)-1])
                break
            else:
                nw += ch
        wor += nw + '^'
    wor = wor[:len(wor)-1]
    return wor, flg, err

def get_ngrams(lines, n):
    """ Give ngrams count """
    t = dict()
    for i in range(0,len(lines)-n+1):
        flg = 0
        for k in range(0,n):
            if lines[i+k] == '.' or lines[i+k] == '':
                flg = 1
        if flg == 1:
            pass
        else:
            x = []
            for j in range(0,n):
                x.append(lines[i+j]) 
            word, flag, error = error_check(x)
            if t.get(word) == None:
                t[word] = 1
            else:
                t[word] += 1
    return t
    
def get_error_ngrams(lines, n):
    t = dict()
    for i in range(0,len(lines)-n+1):
        flg = 0
        for k in range(0,n):
            if lines[i+k] == '.':
                flg = 1
        if lines[i] == '':
            flg = 1
        if flg == 1:
            pass
        else:
            x = []
            for j in range(0,n):
                x.append(lines[i+j])
            word, flag, error = error_check(x)
            if len(error) == 0:
                error.append("Correct")
            for m in error:
                if not m in t.keys():
                    t[m] = []
                t[m].append(word)
    return t


def get_count(ngram):
    n = dict()
    for key, value in ngram.items():
        t = dict()
        for i in value:
            if t.get(i) == None:
                t[i] = 1
            else:
                t[i] += 1
        n[key] = t
    return n
            
def get_probs(ngram):
    n = dict()
    for key, value in ngram.items():
        t = dict()
        sum1 = 0
        for i,j in value.items():
            sum1 += j
        for i,j in value.items():
            t[i] = float(j)/sum1
        n[key] = t
    return n

           

unigrams_error = get_error_ngrams(lines, 1)
unigrams_count = get_count(unigrams_error)
unigrams_probs = get_probs(unigrams_count)
# pp.pprint(unigrams_probs)

# trigrams_error = get_error_ngrams(lines, 3)
# trigrams_count = get_count(trigrams_error)
# trigrams_probs = get_probs(trigrams_count)
# pp.pprint(trigrams_probs)


    
        

## Testing

### Reading "dev.txt" and finding P(class|word) using (P(word|class) * P(class)) / P(word)

In [34]:
file = "../Data/dev.txt"
with open(file) as f:
    lines_test = f.read().split("\n")

lines_test = lines_test[:1000]
unigrams_c = get_ngrams(lines_test, 1)
unigrams_count_sort = sorted(unigrams_c.items(), key = lambda x: x[1], reverse = True)
# pp.pprint(unigrams_count_sort)


def class_probs(ngrams_count):
    t = dict()
    total_words = 0
    for key, value in ngrams_count.items():
        for i,j in value.items():
            total_words += j
            
    for key, value in ngrams_count.items():
        class_words = 0
        for i,j in value.items():
            class_words += j
        if t.get(key) == None:
            t[key] = float(class_words)/total_words
    return t

def class_count(ngrams_count):
    t = dict()
    for key, value in ngrams_count.items():
        class_words = 0
        for i,j in value.items():
            class_words += j
        t[key] = []
        t[key] = class_words
    return t

class_probs_1 = class_probs(unigrams_count)
class_count_1 = class_count(unigrams_count)

def get_total_ngrams(ngrams_count_sort):
    total = 0
    for i in ngrams_count_sort:
        total += i[1]
    return total

    
def word_probs(ngrams_count_sort):
    total = 0
    for i in ngrams_count_sort:
        total += i[1]
    t = dict()
    for i in ngrams_count_sort:
        t[i[0]] = float(i[1])/get_total_ngrams(ngrams_count_sort)
    return t
    

word_probs_1 = word_probs(unigrams_count_sort)
# pp.pprint(class_count_1)
# pp.pprint(word_probs_1)

def class_word(word, n, delta):
    temp = []
    prob = 1
    probs = []
    if n == 1:
        ngrams_probs = unigrams_probs
        class_probs_n = class_probs_1
        word_probs_n = word_probs_1
        ngrams_count_sort = unigrams_count_sort
        class_count_n = class_count_1
    
    
    if word_probs_n.get(word) == None:
        word_probs_n[word] = float(1) / get_total_ngrams(ngrams_count_sort)
    for key, value in ngrams_probs.items():
        flg = 0
        class_words = 0 
        for i,j in value.items():
            if word == i:
                prob = j
                prob *= class_probs_n[key]
                prob = float(prob) / word_probs_n[word]
                flg = 1
                break
            else:
                pass
#Laplace smoothing            
        if flg == 0:
            prob = 1
            for i,j in class_count_n.items():
                prob = float(1) / j 
                prob *= class_probs_n[i]
            prob = prob / word_probs_n[word]
                
        probs.append((key, prob))
    probs = sorted(probs, key = lambda x: x[1])
    for i in probs:
        if i[1] > delta:
                temp.append(i)
        else:
            pass
    
    return temp

def get_test_sentences(lines_test):
    text = ""
    for i in lines_test:
        text = text + i + " "
    sentences = text.split(".")
    return sentences

def class_sentence(sentences, n):
    t = dict()
    prob = 1
    for i in sentences:
        t[i] = []
        temp = -10000
        index = ""
        words = i.split(" ")
        words = get_ngrams(words, n)
        errors = []
        for j in words:
            classes = class_word(j, n, 0)
#             print(classes)
            temp_prob = -2
            ind = ""
            for k in classes:
                if k[0] != "Correct":
                    if k[1] > temp_prob:
                        temp_prob = k[1]
                        ind = k[0]
                else:
                    pass
            errors.append((ind, temp_prob))
        t[i] = []
        errors = sorted(errors, key = lambda x: x[1], reverse = True)
        for l in range(0,3):
            t[i].append(errors[l])
            
    return t
        

sentences = get_test_sentences(lines_test)
# print(sentences[1])
sentence_class = class_sentence(sentences, 1)
for key, value in sentence_class.items():
    pp.pprint(key)
    pp.pprint(value)
    

                    
                



    


('the life expectancy of individuals has increased due to better accessibility '
 'to healthcare services and facilities than before ')
[ ('Vt', 0.05147859091149103),
  ('ArtOrDet', 0.04850588354899648),
  ('Nn', 0.021751517286545504)]
('  in addition , a better health care system will allow the people to stay '
 'healthier and prevent some major illness from happening like stroke and '
 'heart attack ')
[ ('ArtOrDet', 0.04850588354899648),
  ('Nn', 0.032627275929818254),
  ('Prep', 0.030089598913054617)]
('  one cause for this is attributed to the era of baby boomers , who are now '
 'mostly in their late fifties or early sixties ')
[ ('ArtOrDet', 0.04850588354899648),
  ('Vt', 0.033932366967010984),
  ('Vt', 0.03346387274853155)]
('  before this solution was implemented , they did a further study , and '
 'realized that the solution will not work ; as the farmers are using '
 'traditional farming methods which are not yield efficient ')
[ ('Mec', 0.17401213829236403),
  ('Rloc-', 0.1